# AFF - Generating Randomized Treated Landscape Scenarios
## Creates Randomized Treatment Scenario Landscapes - exports as ee.Images into an ee.ImageCollection

In [1]:
import ee
import geemap
import os
import random
import numpy as np
import math
from datetime import datetime
ee.Initialize()

## USER INPUT - Review/edit paths to your AOIs

In [2]:
parcels = ee.FeatureCollection("projects/aff-treatments/assets/AllParcels_forSIG")
NIparcels = ee.FeatureCollection("projects/aff-treatments/assets/AllNIPParcels_forSIG")
fireshed = ee.FeatureCollection("projects/aff-treatments/assets/SonoraFireshed")
parcels_dissolve = ee.FeatureCollection("projects/aff-treatments/assets/NIPparcels_dissolve")

aoi = fireshed

## USER INPUT - Review/edit the parameters for the treatment randomization procedure

In [3]:
TOTAL_AREA = 245278 #acres of AOI

# range of pct of total areas you want to simulate in scenarios
SCENARIO_PCT_RANGE = [0.05,0.6] # make as static list ascending not randomized b/w range

# total scenarios to run
SCENARIOS = 60 

# total number of distinct treatment sizes to use in the scenarios
#TRT_SIZE_CLASSES = 3 # currently this is not dynamic, either make dynamic or take out as a User input

# distinct acreage units to generate
SIZE_CLASSES = [10,40,100,400]  # must be length 4
                               

# LANDFIRE DIST CODE to assign to all treated pixels
DIST_CODE = 222

DISTRO = 'log' # 'log' or 'norm'

## Required Functions, Run and Continue

In [15]:
random.seed(8)

# base python math functions
def randn(low,i_sum,k):
    """Generate k number of floating point numbers adding up to i_sum, with no sample being below low """
    a = np.random.rand(k) #random floats from uniform distribution
    weights = (a/a.sum()*(i_sum-low*k)) # adjust floats to sum to i_sum, with given low val threshold
    adjusted = weights+low # sum(adjusted) should == i_sum
    return [round(i,4) for i in list(adjusted)]

def treatment_math(TOTAL_AREA,DISTRO,SCENARIO_PCT_RANGE,SCENARIOS,SIZE_CLASSES):
    """Returns the amount of treatment units needed per size class and the radii of the kernel needed for each size class to make the randomized treatment landscape in EE
    # args:
    # TOTAL_AREA (int): total area of aoi to consider treatments in acres
    # DISTRO (str): distribution mode to use, one of: 'log', 'norm'
    # SCENARIO_PCT_RANGE (list): min and max range of possible pct treated area floats to be randomly chosen per scenario
    # SCENARIOS (int): Total unique scenarios to run
    # SIZE_CLASSES (list): distinct acreage sizes to generate - list must be of length 4
    """
    dist_dct = {'log': [0.25,0.15,0.05,0.01], # probabilities of the given SIZE_CLASSES for each defined statistical distribution of treatments with acreages ranging 0-400
                'norm': [0.25,0.5,0.05,0.01]}
    
    # make list of length SCENARIOS with random floating point numbers in the range defined by SCNEARIO_PCT_RANGE (list of different total pct treated scenarios)
    scn=[]
    trt_areas=[]
    trt_props=[]
    sizes=[]
    units=[]
    radii=[]
    
    for i in list(range(SCENARIOS)):
        rnd = round(random.uniform(SCENARIO_PCT_RANGE[0], SCENARIO_PCT_RANGE[1]), 4)
        scn.append(rnd)

        # compute total area to be treated for each scenario
        trt_areas_i = TOTAL_AREA*rnd#scn[i]
        trt_areas.append(trt_areas_i)
        
        pdf_probs = dist_dct[DISTRO] # grab the probabilities assigned to each tretment size class from the distribution dictionary
        
        i_prop = [round(p/sum(pdf_probs),4) for p in pdf_probs] # normalize the pdf_probs floats so they sum to 1, so we can use them as percentage of total acreages to treat
        i_prop = [round(p*trt_areas_i,4) for p in i_prop] # get acreage per size class as (pct of size class in distribution * total area to be treated)
        trt_props.insert(i,i_prop)
                
        units_i = [int(round(j)) for j in list(np.divide(i_prop,SIZE_CLASSES)) ]
        units.insert(i,units_i)
        
        # get approximate radius in meters needed for each size class to make correct-sized treatment units
        acres_to_sqm = [int(round(size_i*4027)) for size_i in SIZE_CLASSES] # convert to sq meters for each size class in acreage
        radii_i = [int(round(math.sqrt(acreage/math.pi))) for acreage in acres_to_sqm] # solve for r: A = pi(r^2)
        radii.insert(i,radii_i)
    
    return scn,trt_areas,trt_props,units,radii

# EE functions
def distanceFilter(pts,distance):
    withinDistance = distance; ##go with pretty far apart at first

    ## From the User Guide: https:#developers.google.com/earth-engine/joins_spatial
    ## add extra filter to eliminate self-matches
    distFilter = ee.Filter.And(ee.Filter.withinDistance(**{
      'distance': withinDistance,
      'leftField': '.geo',
      'rightField': '.geo', 
      'maxError': 1
    }), ee.Filter.notEquals(**{
      'leftField': 'system:index',
      'rightField': 'system:index',

    }));
    
    distSaveAll = ee.Join.saveAll(**{
                  'matchesKey': 'points',
                  'measureKey': 'distance'
    });
    # Apply the join.
    spatialJoined = distSaveAll.apply(pts, pts, distFilter);

    # Check the number of matches.
    # We're only interested if nmatches > 0.
    spatialJoined = spatialJoined.map(lambda f: f.set('nmatches', ee.List(f.get('points')).size()) );
    spatialJoined = spatialJoined.filterMetadata('nmatches', 'greater_than', 0);

    # The real matches are only half the total, because if p1.withinDistance(p2) then p2.withinDistance(p1)
    # Use some iterative logic to clean up
    def unpack(l): 
        return ee.List(l).map(lambda f: ee.Feature(f).id())

    def iterator_f(f,list):
        key = ee.Feature(f).id()
        list = ee.Algorithms.If(ee.List(list).contains(key), list, ee.List(list).cat(unpack(ee.List(f.get('points')))))
        return list
    
    ids = spatialJoined.iterate(iterator_f,ee.List([]))
    ##print("Removal candidates' IDs", ids);

    # Clean up 
    cleaned_pts = pts.filter(ee.Filter.inList('system:index', ids).Not());
    return cleaned_pts

def ee_treatments(units,radii):
    units = list(reversed(units)) # we generate treatment units in descending order of size
    radii = list(reversed(radii))
    
    seed = 5155115
    ptsBiggest = ee.Image.constant(1).clip(aoi).sample(aoi,30,'EPSG:3857',None,units[0]+(units[0]*2),seed,True,4,True) 
    ptsBiggest = distanceFilter(ptsBiggest,radii[0]*2.1).limit(units[0])
#     print('Biggest trt points',ptsBiggest.size().getInfo())
    areasBiggest = ptsBiggest.reduceToImage(['constant'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[0],'meters')).eq(1)

    areasBiggestmask = areasBiggest.distance(ee.Kernel.euclidean(radii[0]*2,'meters')).gte(0).Not().unmask(1)
    
    
    ptsBig = ee.Image.constant(1).clip(aoi).sample(aoi,30,'EPSG:3857',None,units[1]+(units[1]*5),seed,True,4,True).map(lambda f: f.set('nd',1)) 
    ptsBig = distanceFilter(ptsBig,radii[1]*2.1).limit(units[1])
#     print('Big trt points',ptsBig.size().getInfo())
    areasBig = ptsBig.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[1],'meters')).eq(1)

    areasBigmask = areasBig.distance(ee.Kernel.euclidean(radii[1]*2,'meters')).gte(0).Not().unmask(1)
    blendmask = areasBiggestmask.add(areasBigmask).lte(1).Not().unmask(1)

#     Map.addLayer(ptsBig,{},'big pts')
#     Map.addLayer(areasBig, pal, 'big treatments')
#     Map.addLayer(areasBigmask,pal,'areasBigmask')
    
    ptsMedium = areasBigmask.selfMask().sample(aoi,30,'EPSG:3857',None,units[2]+(units[2]*5),seed,True,4,True).map(lambda f: f.set('nd',1))
    ptsMedium = distanceFilter(ptsMedium,radii[2]*2.1).limit(units[2])
#     print('Medium trt points',ptsMedium.size().getInfo())
    areasMedium = ptsMedium.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[2],'meters')).eq(1)

    areasMediummask = areasMedium.distance(ee.Kernel.euclidean(radii[2]*2,'meters')).gte(0).Not().unmask(1)
    blend2mask = areasBigmask.add(areasMediummask).lte(1).Not().unmask(1)


    
    ptsSmall = blend2mask.selfMask().sample(aoi,30,'EPSG:3857',None,units[2]+(units[3]*10),seed,True,4,True).map(lambda f: f.set('nd',1))
    ptsSmall = distanceFilter(ptsSmall,radii[3]*2.1).limit(units[3])
#     print('Small trt points',ptsSmall.size().getInfo())
    areasSmall = ptsSmall.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[3],'meters')).eq(1)
    
#     Map.addLayer(ptsSmall, {}, 'small points')
#     Map.addLayer(areasSmall, pal, 'small treatments')
    
    blendTreatments = areasBiggest.add(areasBig).add(areasMedium).add(areasSmall).gte(1).multiply(DIST_CODE).selfMask().rename('DIST')
    
#     Map.addLayer(blendTreatments, pal,'final treatment landscape')

    return ptsBiggest,areasBiggest,areasBiggestmask,ptsBig,areasBig,blendmask,ptsMedium,areasMedium,blend2mask,ptsSmall,areasSmall,ee.Image(blendTreatments)

def export_img(img,imgcoll_p,aoi):
    """Export image to imageCollection"""
    desc = f"scenario{ee.String(ee.Image(img).getNumber('scenario').format()).getInfo()}"
    
    task = ee.batch.Export.image.toAsset(
        image=ee.Image(img),
        description=desc,
        assetId=f'{imgcoll_p}/{desc}', 
        region=aoi.geometry().bounds(), 
        scale=30, 
        crs='EPSG:5070', 
        maxPixels=1e13)

    task.start()
    print(f"Export Started for {imgcoll_p}/{desc}")
    

## Construct random treatment landscapes as rasters - format: an ee.Image exported to an ee.ImageCollection

In [ ]:
Map = geemap.Map()
pal = {'min':0,'max':1,'palette':['black','white']}


# Run treatment math to construct your lists (length of SCENARIO) of the needed parameters
scn,trt_areas,trt_props,units,radii = treatment_math(TOTAL_AREA,DISTRO,SCENARIO_PCT_RANGE,SCENARIOS,SIZE_CLASSES)

print(f'Static Parameters: DISTRO = {DISTRO}; SIZE_CLASSES (ac) = {SIZE_CLASSES}; TOTAL_AREA (ac) = {TOTAL_AREA}')
print('First randomly generated scenario')
print('scenario pct area: ',scn[0])
print('total area (ac) to treat: ', trt_areas[0])    
print('area per size class: ',trt_props[0])
print('units per size class: ', units[0])
print('radii per trt size: ',radii[0])
print('\n')

# for each scenario, make the treated landscape raster 
today_string = datetime.utcnow().strftime("%Y-%m-%d").replace("-", "")

# make an ee.ImageCollection with specified path
img_coll_p = f"projects/aff-treatments/assets/scenarios_{SCENARIOS}_{today_string}"
#os.popen(f"earthengine create collection {img_coll_p}").read()

# trt_imgs = []
# for i in list(range(SCENARIOS)):

ptsBiggest,areasBiggest,areasBiggestmask,ptsBig,areasBig,blendmask,ptsMedium,areasMedium,blend2mask,ptsSmall,areasSmall,trt_img = ee_treatments(units[0],radii[0])#.set('scenario',i+1)



print('Biggest trt points',ptsBiggest.size().getInfo())
print('Big trt points',ptsBig.size().getInfo())
# Map.addLayer(ptsBig,{},'big pts')
Map.addLayer(areasBiggest, pal, 'biggest treatments')
Map.addLayer(areasBiggestmask,pal,'areasBiggestmask')

Map.addLayer(areasBig, pal, 'big treatments')
Map.addLayer(blendmask,pal,'blendmask')

print('Medium trt points',ptsMedium.size().getInfo())
print('Small trt points',ptsSmall.size().getInfo())

Map.addLayer(areasMedium,{},'Medium treatments')
Map.addLayer(blend2mask,pal,'blend2mask')

# Map.addLayer(ptsSmall, {}, 'small points')

Map.addLayer(areasSmall, pal, 'small treatments')
Map.addLayer(trt_img, pal,'final treatment landscape')
    
    
    
    
    #export_img(trt_img,img_coll_p,aoi) #export image to image collection
    #break



# # making one works..
#trt_img_example = ee_treatments(units[5],radii[5]).set('scenario',5+1)
# print(trt_img.getInfo())

Static Parameters: DISTRO = log; SIZE_CLASSES (ac) = [10, 40, 100, 400]; TOTAL_AREA (ac) = 245278
First randomly generated scenario
scenario pct area:  0.1747
total area (ac) to treat:  42850.0666
area per size class:  [23289.0112, 13973.4067, 4657.8022, 929.8464]
units per size class:  [2329, 349, 47, 2]
radii per trt size:  [113, 226, 358, 716]


Biggest trt points 2
Big trt points 47
Medium trt points 349
Small trt points 2329


In [ ]:
#Map = geemap.Map()

# Map.addLayer(parcels,{},'parcels')
# Map.addLayer(NIparcels,{},'NIP parcels')
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi,12)
pal = {'min':0,'max':1,'palette':['black','white']}

#print(trt_imgs.first().propertyNames().getInfo())
# Map.addLayer(trt_imgs.sort('scenario',False).first(), pal, 'final trt landscapes')
#Map.addLayer(trt_img, pal, 'first scenario')
#Map.addLayer(trt_img_example, pal, 'other random scneario')



Map

In [ ]:

print('scenario pct area: ',scn[15])
print('total area (ac) to treat: ', trt_areas[15])    
print('area per size class: ',trt_props[15])
print('sizes: ', sizes[15])
print('units per size class: ', units[15])
print('radii per trt size: ',radii[15])
print('\n')